In [12]:
import os
import random
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image

In [13]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
from tensorflow.keras import backend as K

In [14]:
#This section loads the data and splits it to training, validation and test dataset
def load_data(path, split=0.2, data_type='hypocotyl'):
  if data_type=='cotyledon':
    label_dir='label_dir_cotyledon/*'
  if data_type=='hypocotyl':
    label_dir='label_dir_hypocotyl/*'
  images = sorted(glob(os.path.join(path, "image_dir/*")))
  masks = sorted(glob(os.path.join(path, label_dir)))

  total_size = len(images)
  valid_size = int(split * total_size)

  #split data so 10% is used as validarion
  train_x, valid_x = train_test_split(images, test_size=valid_size, random_state=42)
  train_y, valid_y = train_test_split(masks, test_size=valid_size, random_state=42)

  return (train_x, train_y), (valid_x, valid_y)
def load_test(path):
  images = sorted(glob(os.path.join(path, "image_dir/*")))
  masks = sorted(glob(os.path.join(path, "label_dir_cotyledon/*")))

  return images, masks



In [15]:
#Speficies how the images should be opened and devided by 255 so that each pixel value goes from 0 to 255
def read_image(path):
     path = path.decode()
     x = cv2.imread(path, cv2.IMREAD_COLOR)
     x = np.array(x)

     x=x/255
     return x
def read_mask(path):
     path = path.decode()
     x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

     x = x/255.0
     x = np.expand_dims(x, axis=-1)
     return x


def read_image_test(path):
     x = cv2.imread(path, cv2.IMREAD_COLOR)
     x = np.array(x)
     x=x/255
     return x
def read_mask_test(path):
     x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
     x = x/255.0
     x = np.expand_dims(x, axis=-1)
     return x


def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
    x.set_shape([1024, 1024, 3])
    y.set_shape([1024, 1024, 1])
    return x, y

def tf_dataset(x, y, batch=1):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset


In [16]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.utils import CustomObjectScope

In [17]:
#This is the convolutional operation that gets implemented at various stages of the U-Net network
def conv_block(x, num_filters):
    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

In [18]:
#Unet model implementation Encoder -> Decoder 
def build_model():
    size = 1024
    num_filters = [32, 48, 64, 128]
    inputs = Input((size, size, 3))

    skip_x = []
    x = inputs

    ## Encoder
    for f in num_filters:
        x = conv_block(x, f)
        skip_x.append(x)
        x = MaxPool2D((2, 2))(x)

    ## Bridge
    x = conv_block(x, num_filters[-1])

    num_filters.reverse()
    skip_x.reverse()

    ## Decoder
    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2))(x)
        xs = skip_x[i]
        x = Concatenate()([x, xs])
        x = conv_block(x, f)

    ## Output
    x = Conv2D(1, (1, 1), padding="same")(x)
    x = Activation("sigmoid")(x)

    return Model(inputs, x)

In [19]:
#Defines a type of validation measurment iou

def dice_coef(y_true, y_pred, smooth=0.001):
    intersection = K.sum(y_true * y_pred, axis=[1, 2, 3])
    union = K.sum(y_true, axis=[1, 2, 3]) + K.sum(y_pred, axis=[1, 2, 3])
    return -K.mean((2.0 * intersection + smooth) / (union + smooth), axis=0)



def iou(y_true, y_pred):
     def f(y_true, y_pred):
         intersection = (y_true * y_pred).sum()
         union = y_true.sum() + y_pred.sum() - intersection
         x = (intersection + 1e-15) / (union + 1e-15)
         x = x.astype(np.float32)
         return x
     return tf.numpy_function(f, [y_true, y_pred], tf.float32)

In [38]:
def train_model():
    
    lr = 1e-4
    epochs = 100
    train_steps=100
    valid_steps=100
    model = build_model()
    opt = tf.keras.optimizers.Adam(lr)
    metrics = ["acc", tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), iou]
    model.compile(loss=[dice_coef], optimizer=opt, metrics=metrics)
    return model


In [21]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard

In [22]:
early_stopping=tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5
)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='weight_output/cot.h5',
                                                 save_weights_only=True,
                                                 verbose=1)


In [41]:

def train_seedling_segmentation(data_dir, class_type,batch_size=6,epochs=120):
    path=data_dir
    batch=batch_size
    number_images=len(os.listdir(os.path.join(path,'image_dir')))
    steps_per_train=int((number_images/batch)*0.8)
    steps_per_val=int((number_images/batch)*0.2)

    #Model training callbacks
    if class_type=='cotyledon':
        model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='weight_output/model_weights_cotyledon.h5',
                                                 save_weights_only=True,
                                                 verbose=1)
    if class_type=='hypocotyl':
        model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='weight_output/model_weights_hypocotyl.h5',
                                                 save_weights_only=True,
                                                 verbose=1)
    early_stopping=tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=5, min_lr=0.0000001)
    (train_x, train_y), (val_x, val_y) = load_data(path, data_type=class_type)
    model=train_model()
    train_dataset=tf_dataset(train_x, train_y, batch=batch)
    val_dataset = tf_dataset(val_x, val_y, batch=batch)
    history=model.fit(train_dataset, validation_data=val_dataset, epochs=epochs, validation_steps=steps_per_val, steps_per_epoch=steps_per_train, callbacks=[model_checkpoint,early_stopping,reduce_lr])
    return history


In [43]:
model_history_cotyledon=train_seedling_segmentation(data_dir='D:/seedling_dataset',class_type='cotyledon')

Epoch 1/120
 225/2836 [=>............................] - ETA: 26:31 - loss: -0.8645 - acc: 0.9541 - recall_2: 0.9592 - precision_2: 0.9955 - iou: 0.7649

KeyboardInterrupt: 

In [42]:
model_history_cotyledon=train_seedling_segmentation(data_dir='D:/seedling_dataset',class_type='hypocotyl')

Epoch 1/120
  61/2836 [..............................] - ETA: 26:16 - loss: -0.7752 - acc: 0.8300 - recall_1: 0.8327 - precision_1: 0.9920 - iou: 0.6372

KeyboardInterrupt: 